In [1]:
%pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 2.1 단어 임베딩 이해하기

- 임베딩(embedding): 단어를 벡터 형태로 변환하는 개념
    - 이산적인 객체를 연속적인 벡터 공간의 한 포인트로 매핑
    - 단어 임베딩이 가장 일반적인 형태이나 RAG의 경우 문장, 단락 임베딩 수행
    - 단어 임베딩의 차원은 하나에서 수천까지 가능
        - 차원이 높을수록 미묘한 관계를 잘 감지하나 계산 효율성이 떨어짐.

- LLM은 일반적으로 입력층의 일부로 자체적인 임베딩을 만들고 훈련 중에 업데이트 함.
    - 모델 내부에서 학습, 훈련 중 계속 업데이트 됨
    - 언어 모델 목표에 최적화 됨
    - 즉 LLM에서 임베딩은 '전처리 단계'가 아닌 모델의 첫 번째 학습 가능한 레이어임.
    - 임베딩 크기를 hidden state 차원으로 부르기도 함.

## 2.2 텍스트 토큰화 하기

In [2]:
with open("train_wiki_ko.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 457067719
제임스 얼 "지미" 카터 주니어 (, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다. 지미 카터는 조지아주 섬터 카운티 플레인


In [3]:
import re
preprocessed = re.split(r'([`,.#:*&^%$><;?!=+"()\'-]|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(len(preprocessed))
print(preprocessed[:30])

128781900
['제임스', '얼', '"', '지미', '"', '카터', '주니어', '(', ',', '1924년', '10월', '1일', '~', ')', '는', '민주당', '출신', '미국', '39번째', '대통령', '(', '1977년', '~', '1981년', ')', '이다', '.', '지미', '카터는', '조지아주']


## 2.3 토큰을 토큰 ID로 변환하기

- 앞서 생성한 토큰을 토큰 ID로 매핑하기 위해서는 어휘사전(vocabulary)을 먼저 구축해야 함
- 어휘사전을 통해 개별 단어와 특수 문자를 정수로 매핑함

In [4]:
all_words = set(preprocessed) # 집합

#print(list(enumerate(all_words))) -> [(0, '사과'), (1, '배'), (2, '고추장')]
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 10:
        break;

('아즈키와의', 0)
('‘폐’', 1)
('놈이', 2)
('달성되었으며', 3)
('무도장이지만', 4)
('제43조와', 5)
('0·100·200번대가', 6)
('정원않고일까', 7)
('승부욕의', 8)
('M1918A1으로', 9)
('鳳山鎭', 10)


In [5]:
class tokenizerV1:
    def __init__(self, vocab):
        self.str2int = vocab
        self.int2str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([`,.#:*&^%$><;?!=+"()\'-]|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str2int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = "".join([self.int2str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [6]:
with open("test_review_movieid.txt", "r", encoding="utf-8") as f:
    test_text = f.read()

tokenizer = tokenizerV1(vocab)

# print(tokenizer.encode(test_text))
# KeyError: '␞92575'

In [7]:
# <|unk|>, <|endoftext|> 추가
# 여러 개의 독립적인 텍스트 소스로 작업할 때에는 텍스트 사이에 <|endoftext|> 토큰을 추가한다.
# <|endoftext|>는 특정 세그먼트의 시작 또는 끝을 알리는 마커로 작동한다.

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [8]:
class tokenizerV2:
    def __init__(self, vocab):
        self.str2int = vocab
        self.int2str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([`,.#:*&^%$><;?!=+"()\'-]|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]

        #추가
        preprocessed = [item if item in self.str2int 
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str2int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = "".join([self.int2str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [9]:
tokenizer = tokenizerV2(vocab)
encoded = tokenizer.encode(test_text)
print(encoded[:40])

[6089864, 6852458, 400825, 3350190, 12, 7413315, 5398303, 6501272, 5593075, 5416230, 2488047, 5748091, 6263671, 5834135, 4826733, 3380599, 12, 5398308, 5195833, 7413315, 3536559, 12, 7413315, 2929540, 2333949, 5710206, 4950890, 3529822, 2292981, 7413315, 3913517, 5137857, 1219598, 2880949, 7413315, 5555391, 3279856, 4049130, 2997015, 7275818]


## 2.5 바이트 페어 인코딩

In [10]:
%pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tiktoken

with open("train_wiki_ko.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

819815817


: 

## 2.6 슬라이딩 윈도우로 데이터 샘플링하기

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class DataSetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]